In [1]:
#ipython magic
#%reset -f
%matplotlib notebook 
%load_ext autoreload
%autoreload 2

In [2]:
#general imports
import sys
sys.path.append('..')
sys.path.append('/home/jamesrowland/Documents/Code/Vape/utils')
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import copy

In [3]:
#notebook specific imports
import pickle
import time
from utils_funcs import s2p_loader, correct_s2p_combined, dfof, dfof2
from matplotlib.widgets import Lasso
from matplotlib.collections import RegularPolyCollection
from matplotlib import colors as mcolors, path

In [4]:
#global plotting params
sns.set()
sns.set_style('white')
params = {'legend.fontsize': 'x-large',
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
plt.rcParams.update(params)


In [20]:
with open('/home/jamesrowland/Documents/Code/Vape/run_pkls/J048/run19.pkl', 'rb') as f:
    run = pickle.load(f)

In [6]:
correct_s2p_combined(run.s2p_path)
combined_path = os.path.join(run.s2p_path, 'combined')

In [7]:
flu, stat = s2p_loader(combined_path, subtract_neuropil=True)
flu = dfof2(flu)
flu.shape

Loading 4244 traces labelled as cells
Subtracting neuropil with a coefficient of 0.7


(4244, 15021)

In [8]:
assert sum(run.num_frames) == flu.shape[1] * 3 # change if not three planes

In [22]:
num_frames = run.num_frames
num_frames

[13548, 13548, 13548, 4419]

In [10]:
fc = run.frame_clock / run.paq_rate

In [51]:
gap_idx = np.where(np.diff(fc) > 1)
gap_idx = np.insert(gap_idx, 0, 0)
gap_idx = np.append(gap_idx, len(fc))

In [52]:
clock_num_frames

array([   99, 13557, 13557, 13558,  4426])

In [69]:
clock_num_frames = np.diff(gap_idx)
real_frames = [False] * len(fc)

prev = 0

for n in num_frames: 

    try:
        idx = next(i for i,c in enumerate(clock_num_frames) if c >= n-15 and c <= n + 15)
        real_frames[prev:prev+clock_num_frames[idx] = True
    except StopIteration:
        continue 
        
    print(clock_num_frames[idx])
    clock_num_frames = np.delete(clock_num_frames, idx)


13557
13557
13558
4426


In [17]:
class Datum(object):
    colorin = mcolors.to_rgba("red")
    colorout = mcolors.to_rgba("blue")

    def __init__(self, x, y, include=False):
        self.x = x
        self.y = y
        if include:
            self.color = self.colorin
        else:
            self.color = self.colorout


class LassoManager(object):
    def __init__(self, ax, data):
        self.axes = ax
        self.canvas = ax.figure.canvas
        self.data = data

        self.Nxy = len(data)

        facecolors = [d.color for d in data]
        self.xys = [(d.x, d.y) for d in data]
        self.collection = RegularPolyCollection(4, sizes=(100,),
                                                facecolors=facecolors,
                                                offsets=self.xys,
                                                transOffset=ax.transData,
                                                alpha =0.01)
        ax.add_collection(self.collection)

        self.cid = self.canvas.mpl_connect('button_press_event', self.onpress)

    def callback(self, verts):
        facecolors = self.collection.get_facecolors()
        p = path.Path(verts)
        # True if point selected else false 
        self.ind = p.contains_points(self.xys)
        for i in range(len(self.xys)):
            if self.ind[i]:
                facecolors[i] = Datum.colorin
            else:
                facecolors[i] = Datum.colorout

        self.canvas.draw_idle()
        self.canvas.widgetlock.release(self.lasso)
        del self.lasso

    def onpress(self, event):

        if self.canvas.widgetlock.locked():
            return
        if event.inaxes is None:
            return
        self.lasso = Lasso(event.inaxes,
                           (event.xdata, event.ydata),
                           self.callback)
        # acquire a lock on the widget drawing
        self.canvas.widgetlock(self.lasso)


axis_edge = 2*min(frame_clock) 
data = [Datum(*xy) for xy in np.transpose(np.vstack((frame_clock, np.ones(len(frame_clock)))))]
ax = plt.axes(xlim=(min(frame_clock)-axis_edge, max(frame_clock)+axis_edge), ylim=(0, 2), autoscale_on=False)
ax.set_title('Lasso points using left mouse button')

lman = LassoManager(ax, data)
plt.show()

NameError: name 'frame_clock' is not defined